In [1]:
import os 

experiments = os.listdir("../experiments")

### Find Best Params

First find the best params

In [2]:
def file_read_from_tail(fname,lines):
        fname = "../experiments/" + fname
        bufsize = 8192
        fsize = os.stat(fname).st_size
        iter = 0
        with open(fname) as f:
                if bufsize > fsize:
                        bufsize = fsize-1
                        data = []
                        while True:
                                iter +=1
                                f.seek(fsize-bufsize*iter)
                                data.extend(f.readlines())
                                if len(data) >= lines or f.tell() == 0:
                                        return data[-lines:]


In [3]:
def parse_mse(line):
    mse = float(line.strip()[4:])
    return mse

In [57]:
from statistics import mean, stdev

def aggregateParams(files):
    mse_to_params = {}
    params_to_mse = {}
    
    # aggregate mse
    for f in files:
        key = f[:-6]
        d = file_read_from_tail(f,3)
        if d == None:
            continue
        mse = parse_mse(d[0])
        
        if key not in params_to_mse:
            params_to_mse[key] = []
        
        params_to_mse[key].append(mse)
                
    # flip and sort mses
    all_mse = []
    for params in params_to_mse:
        # get means and standard deviations
        mean_mse = mean(params_to_mse[params])
        std_mse = stdev(params_to_mse[params])
        # reassign
        params_to_mse[params] = (mean_mse, std_mse)
        # sort means
        all_mse.append(mean_mse)
        mse_to_params[mean_mse] = params
                 
    all_mse.sort()

    # return top 5 mses
    results = []    
    for i in range(20):
        params = mse_to_params[all_mse[i]]
        (mean_mse, std_mse) = params_to_mse[params]
        print("Result " + str(i) + ": " +  str(params) + " with mean mse " + str(mean_mse) + ", std mse " + str(std_mse))
        results.append(params)
            
    return results, params_to_mse, mse_to_params, all_mse

In [58]:
results, params_to_mse, mse_to_params, all_mse = aggregateParams(experiments)

Result 0: maxtime20_pop512_mr0.001_tour8_maxHeight2_cr1 with mean mse 4324.2002999999995, std mse 482.374113327232
Result 1: maxtime15_pop512_mr0.1_tour8_maxHeight2_cr1 with mean mse 4377.521, std mse 693.5872031814823
Result 2: maxtime20_pop512_mr0.01_tour8_maxHeight2_cr1 with mean mse 4411.3274, std mse 531.7482366106058
Result 3: maxtime20_pop256_mr0.01_tour8_maxHeight4_cr1 with mean mse 4471.228, std mse 648.0514775324566
Result 4: maxtime20_pop1024_mr0.01_tour8_maxHeight2_cr1 with mean mse 4512.0114, std mse 344.5601731893381
Result 5: maxtime20_pop1024_mr0_tour8_maxHeight2_cr1 with mean mse 4553.743, std mse 651.7048964724925
Result 6: maxtime20_pop1024_mr0.1_tour8_maxHeight2_cr0.75 with mean mse 4554.3984, std mse 511.43883018080584
Result 7: maxtime15_pop256_mr0.1_tour8_maxHeight2_cr0.75 with mean mse 4558.7931, std mse 722.5538271608327
Result 8: maxtime20_pop1024_mr0.1_tour8_maxHeight2_cr1 with mean mse 4579.0179, std mse 615.8677080512682
Result 9: maxtime20_pop1024_mr0.01_t

### Analyse Simple GP Behaviour

Holding all other variables constant, lets see what affect the params have